In [ ]:
# ==========================
# 1️⃣ 라이브러리 임포트
# ==========================
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ==========================
# 파일 경로
# ==========================
DATA_DIR = "C:/ESG_Project1/file/merge_data"
train_path = os.path.join(DATA_DIR, "train_data.csv")
test_path  = os.path.join(DATA_DIR, "test_data.csv")

# ==========================
# 2️⃣ 데이터 로드
# ==========================
train_df = pd.read_csv(train_path, index_col=0, parse_dates=True)
test_df  = pd.read_csv(test_path, index_col=0, parse_dates=True)

target_col = '합산발전량(MWh)'  # XGBoost와 동일

train_values = train_df[[target_col]].values
test_values  = test_df[[target_col]].values

# ==========================
# 3️⃣ 정규화
# ==========================
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_values)
test_scaled  = scaler.transform(test_values)

# ==========================
# 4️⃣ 시퀀스 생성
# ==========================
def create_sequences(data, input_steps=48, output_steps=12):
    X, y = [], []
    for i in range(len(data) - input_steps - output_steps + 1):
        X.append(data[i:i+input_steps])
        y.append(data[i+input_steps:i+input_steps+output_steps])
    return np.array(X), np.array(y)

input_steps = 48   # 2일치 시계열
output_steps = 12  # 12시간 단위 예측
X_train, y_train = create_sequences(train_scaled, input_steps, output_steps)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# ==========================
# 5️⃣ 최적화된 CNN-LSTM 모델
# ==========================
model = Sequential([
    Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(input_steps, 1)),
    Dropout(0.3),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    LSTM(64, activation='tanh'),
    Dropout(0.2),
    Dense(output_steps)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# ==========================
# 6️⃣ 학습
# ==========================
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=150,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# ==========================
# 7️⃣ Rolling Forecast 예측
# ==========================
rolling_input = list(train_scaled[-input_steps:])  # 마지막 학습 데이터 기준
predicted_scaled = []

for i in range(len(test_scaled)):
    X_input = np.array(rolling_input[-input_steps:]).reshape(1, input_steps, 1)
    y_pred_step = model.predict(X_input, verbose=0)
    next_pred = y_pred_step[0][0]  # 첫 시점 예측값
    predicted_scaled.append(next_pred)
    rolling_input.append([next_pred])

predicted_scaled = np.array(predicted_scaled).reshape(-1,1)
predicted_generation = scaler.inverse_transform(predicted_scaled)

# ==========================
# 8️⃣ 성능 평가
# ==========================
y_true = test_values
rmse = np.sqrt(mean_squared_error(y_true, predicted_generation))
mae  = mean_absolute_error(y_true, predicted_generation)
r2   = r2_score(y_true, predicted_generation)

print(f"CNN-LSTM 성능")
print(f"MAE : {mae:.2f}, RMSE : {rmse:.2f}, R² : {r2:.4f}")

# ==========================
# 9️⃣ 시각화
# ==========================
plt.figure(figsize=(12,4))
plt.plot(test_df.index, y_true, label="실제")
plt.plot(test_df.index, predicted_generation, label="CNN-LSTM 예측")
plt.xlabel("시간")
plt.ylabel("발전량 (MWh)")
plt.title("CNN-LSTM 실제 vs 예측")
plt.legend()
plt.grid(True)
plt.show()
